# 01/20

%matplotlib inline

In [1]:
description = 'train_mobilenet_from_pretrain_v3_norm_val_aro_all_data'

In [2]:

import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import cv2
from PIL import Image
from random import shuffle
import math

import pandas as pd

import pickle

from sklearn.svm import SVC,LinearSVC
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier,ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing

#from scipy.misc import imread, imresize



In [3]:
import tqdm
import json

In [4]:
train_df = pd.read_csv('../../data/Manually_Annotated_file_lists/training_face_mesh_crop.csv')
train_df.subDirectory_filePath = '../../data/Manually_Annotated_Images_FaceMesh_Cropped/' + train_df.subDirectory_filePath

train_df_2 = pd.read_csv('../../data/Automatically_annotated_file_list/automatically_annotated_face_mesh_crop.csv')
train_df_2.subDirectory_filePath = '../../data/Automatically_Annotated_Images_FaceMesh_Cropped/' + train_df_2.subDirectory_filePath
train_df = train_df.append(train_df_2)
del train_df_2

train_df = train_df[train_df['have_facemesh']]

In [5]:
def scale(x):
    if x > -1 and x < 1:
        return x
    elif x <= -1:
        return -1
    else:
        return 1

In [6]:
train_df.valence = train_df.valence.apply(scale)
train_df.arousal = train_df.arousal.apply(scale)

In [7]:
from eason_utils import DataFrameBatchIterator
from eason_utils import lprint, now_time_string, log_file_name, change_log_file_name

change_log_file_name(f'''{log_file_name.replace('.log', '')}_{description}.log''')

log_file_name='01_22_23:59:34.log'


In [8]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
config = tf.compat.v1.ConfigProto(gpu_options = tf.compat.v1.GPUOptions(allow_growth = True))
sess = tf.compat.v1.Session(config = config)

In [9]:
mobilenet_pretrained = tf.keras.models.load_model('../models/affectnet_emotions/mobilenet_7.h5')

In [10]:
mobilenet_output = mobilenet_pretrained.get_layer("global_pooling").output

In [11]:
hidden_layers = [256]

In [12]:
valaence_feat = mobilenet_output
for size in hidden_layers:
    valence_feat = tf.keras.layers.Dense(size, activation = 'relu', name = f'feat_valence')(valaence_feat)
outputs_valence = (tf.keras.layers.Dense(1, activation = 'sigmoid', name = 'outputs_valence')(valence_feat) * 2) - 1

arousal_feat = mobilenet_output
for size in hidden_layers:
    arousal_feat = tf.keras.layers.Dense(size, activation = 'relu', name = f'feat_arousal')(arousal_feat)
outputs_arousal = (tf.keras.layers.Dense(1, activation = 'sigmoid', name = 'outputs_arousal')(arousal_feat) * 2) - 1

emotion_feat = mobilenet_output
for size in hidden_layers:
    emotion_feat = tf.keras.layers.Dense(size, activation = 'relu', name = f'feat_emotion')(emotion_feat)
emotions_count = len(np.unique(train_df.expression))
outputs_emotion = tf.keras.layers.Dense(emotions_count, activation = 'softmax', name = 'outputs_emotion')(emotion_feat)

In [13]:
model = tf.keras.Model(inputs=mobilenet_pretrained.input,
                       outputs=(outputs_valence, outputs_arousal, outputs_emotion) , name="mobilenet_train")

In [14]:
model.summary()

Model: "mobilenet_train"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 225, 225, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1 (Conv2D)                 (None, 112, 112, 32  864         ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                    

In [15]:
# Instantiate an optimizer.
optimizer = tf.keras.optimizers.Adam()
# Instantiate a loss function.
MSE_loss = tf.keras.losses.MeanSquaredError()
SCC_loss = tf.keras.losses.SparseCategoricalCrossentropy()

In [16]:
batch_size = 32
epochs = 16

logs = []

In [ ]:
from telegram_notifier import send_message
import time
start_time = time.time()


for epoch in range(epochs):
    total_loss = 0
    total_step = 0
    total_emotion_correct = 0
    total_valence_loss = 0
    total_arousal_loss = 0
    total_emotion_loss = 0
    total_process = 0
    for step, row in enumerate(DataFrameBatchIterator(train_df, batch_size=batch_size)):

        imgs = row.subDirectory_filePath.apply(lambda x: cv2.resize(
            cv2.cvtColor(cv2.imread(x), cv2.COLOR_BGR2RGB), (224, 224)))
        img_array = np.array(list(imgs))

        y_valence = np.array(row.valence)
        y_arousal = np.array(row.arousal)
        y_emotion = np.array(row.expression)

        with tf.GradientTape() as tape:
            logits = model(img_array, training=True)

            pred_valence = logits[0]
            pred_arousal = logits[1]
            pred_emotion = logits[2]

            valence_loss = MSE_loss(y_valence, pred_valence)
            arousal_loss = MSE_loss(y_arousal, pred_arousal)
            emotion_loss = SCC_loss(y_emotion, pred_emotion)

            loss = valence_loss + arousal_loss + emotion_loss

            grads = tape.gradient(loss, model.trainable_weights)

            optimizer.apply_gradients(zip(grads, model.trainable_weights))
        total_loss += float(loss)
        total_step += 1
        
        total_process += len(row)

        valence_loss = float(valence_loss)
        arousal_loss = float(arousal_loss)
        emotion_loss = float(emotion_loss)
        
        total_valence_loss += valence_loss
        total_arousal_loss += arousal_loss
        total_emotion_loss += emotion_loss
        
        emotion_correct = int(sum(pred_emotion.numpy().argmax(axis = 1) == y_emotion))
        total_emotion_correct += emotion_correct
        
        log = {
            'emotion_correct': emotion_correct,
            'valence_loss': valence_loss,
            'arousal_loss': arousal_loss,
            'emotion_loss': emotion_loss,
            'time_take': time.time() - start_time
        }
        log = json.dumps(log)
        lprint(log)
    save_model_path = f"models/{now_time_string}_{description}_epoch{epoch}_batch_{batch_size}"
    model.save(save_model_path)
    lprint(f"Save {save_model_path}")
    send_message(f"Save {save_model_path}\nepoch {epoch} is finish")
    log = {
        'model_path': save_model_path,
        'total_emotion_correct': total_emotion_correct,
        'total_loss': total_loss,
        'total_valence_loss': total_valence_loss,
        'total_arousal_loss': total_arousal_loss,
        'total_emotion_loss': total_emotion_loss,
        'total_process': total_process,
        'time_take': time.time() - start_time
    }
    log = json.dumps(log)
    lprint(log)

{"emotion_correct": 5, "valence_loss": 0.5505444407463074, "arousal_loss": 0.6094990968704224, "emotion_loss": 2.3576998710632324, "time_take": 5.65541934967041}
{"emotion_correct": 12, "valence_loss": 0.7877877950668335, "arousal_loss": 0.5867432951927185, "emotion_loss": 2.0556793212890625, "time_take": 6.241375684738159}
{"emotion_correct": 14, "valence_loss": 0.8061977028846741, "arousal_loss": 0.55048668384552, "emotion_loss": 1.747434377670288, "time_take": 6.734381914138794}
{"emotion_correct": 19, "valence_loss": 0.6549259424209595, "arousal_loss": 0.49318021535873413, "emotion_loss": 1.6551189422607422, "time_take": 7.287632465362549}
{"emotion_correct": 19, "valence_loss": 0.49637120962142944, "arousal_loss": 0.2615598142147064, "emotion_loss": 1.4801034927368164, "time_take": 7.929156541824341}
{"emotion_correct": 17, "valence_loss": 0.42384690046310425, "arousal_loss": 0.34317460656166077, "emotion_loss": 1.5339140892028809, "time_take": 8.274306058883667}
{"emotion_correct